In [5]:
!pip install kaggle


In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c cifar-10

cifar-10.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!ls


cifar-10.zip  kaggle.json  sample_data


In [9]:
from zipfile import ZipFile
file_name = "cifar-10.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [10]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 31.3 MB/s eta 0:00:00


In [11]:
import py7zr
with py7zr.SevenZipFile('train.7z', mode='r') as z:
    z.extractall()

Import dependencies

In [12]:

import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mp
from sklearn.model_selection import train_test_split


In [13]:
label_df=pd.read_csv('trainLabels.csv')
label_df.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [14]:
label_df.shape

(50000, 2)

In [15]:
label_df['label'].value_counts()

,count
label,
frog,5000
truck,5000
deer,5000
automobile,5000
bird,5000
horse,5000
ship,5000
cat,5000
dog,5000


In [16]:
labels_dict={"airplane":0,"automobile":1,"frog":2,"truck":3,"deer":4,"bird":5,"horse":6,"ship":7,"cat":8,"dog":9}

In [17]:
labels=[labels_dict[i] for i in label_df['label']]

In [18]:
import cv2
from google.colab.patches import cv2_imshow
img=cv2.imread('/content/train/7796.png')

In [19]:
id_list=list(label_df['id'])

In [20]:
train_data_folder='/content/train/'
data=[]
for i in id_list:
  img=cv2.imread(train_data_folder+str(i)+'.png')
  data.append(img)

In [21]:
X=np.array(data)
y=np.array(labels)

In [22]:
print(X.shape)
print(y.shape)

(50000, 32, 32, 3)
(50000,)


In [23]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [24]:
len(Y_train)

40000

In [25]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [26]:
import tensorflow as tf
from tensorflow import keras

In [27]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(32,32,3)))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [29]:
# model.fit(X_train_scaled,Y_train,epochs=10)

In [30]:
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

In [31]:
conv=ResNet50(include_top=False,input_shape=(256,256,3))
conv.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [32]:
num_classes=10
model=models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(conv)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_classes,activation='softmax'))

In [33]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5),loss='sparse_categorical_crossentropy',metrics=['accuracy']) # Changed lr to learning_rate

In [34]:
history=model.fit(X_train_scaled,Y_train,epochs=10,validation_split=0.1)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 435s 334ms/step - accuracy: 0.3500 - loss: 1.9755 - val_accuracy: 0.8112 - val_loss: 0.7104
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 407s 333ms/step - accuracy: 0.7128 - loss: 0.9684 - val_accuracy: 0.9118 - val_loss: 0.4068
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 382s 333ms/step - accuracy: 0.8300 - loss: 0.6874 - val_accuracy: 0.9310 - val_loss: 0.2954
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 384s 341ms/step - accuracy: 0.8874 - loss: 0.5092 - val_accuracy: 0.9425 - val_loss: 0.2579
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 432s 333ms/step - accuracy: 0.9202 - loss: 0.4001 - val_accuracy: 0.9450 - val_loss: 0.2280
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 382s 333ms/step - accuracy: 0.9461 - loss: 0.3132 - val_accuracy: 0.9498 - val_loss: 0.2023
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 391s 341ms/step - accuracy: 0.9594 - loss: 0.2554 - val_accuracy: 0.9503 - val_loss: 0.1903
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 432s 332ms/step - ac